In [4]:
import gensim
import os, math
import numpy as np
import pandas as pd
import seaborn as sns
import helpers
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
X_train, X_test, y_train, y_test = helpers.train_test_split('./data/training_variants', './data/training_text')

Will mostly follow examples from
http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
and
https://codesachin.wordpress.com/2015/10/09/generating-a-word2vec-model-from-a-block-of-text-using-gensim-python/

In [13]:
unique_text = X_train.Text.unique()

In [151]:
def build_sentences(text):
    sentences = []
    for document in text:
        sts = document.split('.') # list of sentences in the document
        for s in sts:
            sentences.append(s.split(' '))
    return sentences

In [177]:
# build words from the unique text
class sentences:
    def __init__(self, txt):
        self.n = len(txt)
        self.i = 0
        self.txt = txt
    def __iter__(self):
        return self
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        self.i += 1
        return self.txt[self.i-1].split(' ')
k = build_sentences(unique_text[0:500])
model = gensim.models.Word2Vec(k, size=100)
#model.wv.vocab

In [179]:
model.similarity('tumor', 'cancer')

0.46584247226638104

In [195]:
def doc2vec(doc):
    words = doc.split(' ')
    N = 0
    v = np.zeros(100)
    for w in words:
        try:
            v+=model[w.rstrip('.')]
            N +=1
        except KeyError:
            pass
    v = (1./N)*v
    return v

In [198]:
doc2vec(unique_text[2])

array([ -8.75072366e-01,   6.81262876e-01,   1.63646727e-01,
        -6.62472303e-02,   3.16447992e-01,   3.17690920e-01,
        -4.16821289e-01,  -1.26504012e-01,  -7.46390552e-01,
         5.62878910e-01,  -1.25126667e+00,  -8.84982316e-01,
        -1.41146815e-01,   3.55916664e-01,   6.89787582e-01,
        -8.28153093e-01,   5.42554148e-01,   1.16406871e-01,
         3.53334772e-01,   1.64851126e-01,   5.86909858e-02,
        -1.44564929e-01,   3.05625681e-01,  -5.69258288e-01,
        -5.41004969e-02,  -2.35740282e-01,   8.05800762e-02,
        -5.51108543e-01,   6.38452994e-02,   4.60677014e-01,
        -1.11976935e-01,  -3.28893528e-01,   4.52132536e-01,
         5.00385182e-01,   1.18736550e-01,   3.50027780e-01,
         3.53953641e-01,  -1.73438528e-01,  -3.40972794e-01,
         1.64358929e-01,   2.68582777e-01,   2.17740401e-01,
        -3.15967371e-01,   5.25950685e-02,   9.89642717e-01,
        -1.39784870e-01,  -4.33217375e-01,   9.87710120e-05,
        -2.68947544e-01,

In [210]:
v1 = doc2vec(unique_text[0])
v2 = doc2vec(unique_text[200])
np.linalg.norm(v1-v2)

1.1160162346721447